# 🔍 AI Visibility Analyzer

**Query Fan-Out + Content Chunking + Similarity Scoring**

## 📋 Istruzioni:
1. Esegui **Cella 1** → aspetta restart (10 sec)
2. Esegui **Cella 2-8** in ordine
3. Configura URL nella **Cella 6**

---

## 📦 Cella 1: Installazione

⚠️ **Il runtime si riavvierà - è normale!**

In [ ]:
import os

!pip uninstall -y numpy pandas scipy scikit-learn -qq 2>/dev/null
!pip install numpy==1.26.4 -q
!pip install --force-reinstall --no-cache-dir pandas==2.2.2 -q
!pip install dspy-ai openai google-generativeai sentence-transformers beautifulsoup4 lxml -q

print('✅ Installazione completata')
print('🔄 Riavvio runtime...')

os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 65.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
missingno 0.5.2 requires scipy, which is not installed.
jaxlib 0.7.2 requires scipy>=1.13, which is not installed.
tensorflow-decision-forests 1.12.0 requires pandas, which is not installed.
bokeh 3.7.3 requires pandas>=1.2, which is not installed.
scs 3.2.9 requires scipy, which is not installed.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, which is not installed.
umap-learn 0.5.9.post2 requires scipy>=1.3.1, which is not installed.
arviz 0.22.0 requires pandas>=2.1.0, which is not installed.
arviz 0.22.0 requires scipy>=1.11.0, which is not installed.
datasets 4.0.0 requires pandas, which is not installed.
cufflinks 0.17.3 requires pandas>=0.19.2, wh

## 🔧 Cella 2: Import

In [ ]:
import dspy
import json
import re
import os
import datetime
from typing import List, Dict, Optional, Any

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from google.colab import userdata
from IPython.display import HTML, display
from google import genai

print(f'✅ Import OK | NumPy: {np.__version__} | Pandas: {pd.__version__}')

✅ Import OK | NumPy: 2.3.4 | Pandas: 2.2.2


## ⚙️ Cella 3: Config

**IMPORTANTE:** Clicca 🔑 e aggiungi `OPENAI_API_KEY`

In [12]:
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print('✅ Gemini Key OK')
except:
    GEMINI_API_KEY = None
    print('❌ GEMINI_API_KEY mancante!')

MIN_QUERIES_SIMPLE = 10
MIN_QUERIES_COMPLEX = 20
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50
SIMILARITY_THRESHOLD = 0.65

GEMINI_MODEL = 'gemini-2.0-flash-exp'
GEMINI_EMBEDDING_MODEL = 'models/text-embedding-004'

ALLOWED_ROUTING_FORMATS = [
    'web_article', 'faq_page', 'how_to_steps', 'comparison_table',
    'buyers_guide', 'checklist', 'product_spec_sheet', 'glossary/definition',
    'pricing_page', 'review_roundup', 'tutorial_video/transcript',
    'podcast_transcript', 'code_samples/docs', 'api_reference',
    'calculator/tool', 'dataset', 'image_gallery', 'map/local_pack',
    'forum/qna', 'pdf_whitepaper', 'case_study', 'press_release',
    'interactive_widget'
]

print('\n✅ Config OK' if GEMINI_API_KEY else '\n❌ Aggiungi GEMINI_API_KEY!')

✅ Gemini Key OK

✅ Config OK


## 🛠️ Cella 4: Utility Functions

In [ ]:
def extract_content_from_url(url):
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=15)
        r.raise_for_status()
        s = BeautifulSoup(r.content, 'lxml')

        for t in s(['script', 'style', 'nav', 'footer', 'aside', 'iframe']):
            t.decompose()

        title = s.find('title')
        title_text = title.get_text(strip=True) if title else 'Untitled'

        main = s.find('main') or s.find('article') or s.find('body')
        if main:
            elems = main.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li', 'td'])
            content = ' '.join([e.get_text(strip=True) for e in elems])
        else:
            content = s.get_text(separator=' ', strip=True)

        content = re.sub(r'\s+', ' ', content).strip()

        return {
            'success': True,
            'title': title_text,
            'content': content,
            'word_count': len(content.split()),
            'url': url
        }
    except Exception as e:
        return {'success': False, 'error': str(e), 'url': url}


def chunk_text(text, size=512, overlap=50):
    sents = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    curr = []
    clen = 0

    for s in sents:
        words = s.split()
        slen = len(words)

        if clen + slen > size and curr:
            chunks.append(' '.join(curr))
            curr = (curr[-overlap:] if len(curr) > overlap else curr) + words
            clen = len(curr)
        else:
            curr.extend(words)
            clen += slen

    if curr:
        chunks.append(' '.join(curr))

    return chunks


print('✅ Utils OK')

✅ Utils OK


In [48]:
# Cella 4.5: Hybrid Chunking (WordLift + Semantic)

def hybrid_chunk_text(text, analyzer, use_grounding=True, url=None):
    """
    Hybrid approach: try Gemini grounding first, fallback to semantic clustering
    """

    if use_grounding and url:
        print('   🌐 Attempting Gemini grounding chunks...')
        try:
            # Use Gemini url_context tool
            response = analyzer.client.models.generate_content(
                model=analyzer.model,
                contents=[f"Analyze and extract key content sections from: {url}"],
                config={"tools": [{"url_context": {}}]}
            )

            grounded_chunks = []
            if hasattr(response.candidates[0], 'grounding_metadata'):
                chunks_data = response.candidates[0].grounding_metadata.grounding_chunks
                for chunk in chunks_data:
                    if hasattr(chunk, 'retrieved_context'):
                        grounded_chunks.append(chunk.retrieved_context.text.strip())

            if grounded_chunks:
                print(f'   ✅ Got {len(grounded_chunks)} grounded chunks from Gemini')

                # Re-chunk if too large
                final_chunks = []
                for gc in grounded_chunks:
                    if len(gc) > 3000:  # ~600 words
                        sub = chunk_text(gc, max_chunk_length=500, overlap_words=20)
                        final_chunks.extend(sub)
                    else:
                        final_chunks.append(gc)

                return final_chunks

        except Exception as e:
            print(f'   ⚠️ Grounding failed: {e}')

    # Fallback: semantic clustering
    print('   🔄 Fallback to semantic clustering...')
    return semantic_chunk_text(text, analyzer, target_chunk_words=400)


def chunk_text(text, max_chunk_length=500, overlap_words=10):
    """WordLift's mechanical chunking (for re-chunking large grounded chunks)"""
    if not text: return []
    sentences = re.split(r'(?<=[.!?])\s+', text.replace('\n', ' '))
    if not sentences: return [text] if text else []

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) > max_chunk_length and current_chunk:
            chunks.append(current_chunk.strip())
            # Overlap: keep last N words
            current_chunk = " ".join(current_chunk.split()[-overlap_words:])
        current_chunk += " " + sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    return [c for c in chunks if c]

## 🧠 Cella 5: DSPy + Analyzer

In [49]:
# Cella 5 - GeminiAnalyzer with Semantic Chunking

from google import genai
from google.genai import types

class QueryFanOutWithFacets(dspy.Signature):
    """Two-step reasoning: identify facets, then generate queries (WordLift approach)"""
    entity_name = dspy.InputField(desc="Main entity/topic")
    current_date = dspy.InputField(desc="Current date for time-aware queries")
    num_queries = dspy.InputField(desc="Number of queries to generate")

    reasoning_about_facets = dspy.OutputField(
        desc="""Identify 3-5 key information facets for this entity:
        - Definitional/Explanatory (core concepts, what/why)
        - Practical/Implementation (how-to, tools, methods, step-by-step)
        - Comparative/Analytical (benefits, drawbacks, alternatives, comparison)
        - Current/Temporal (recent trends, updates as of current_date)
        - Related/Adjacent (sub-topics, related concepts, ecosystem)

        Distribution target: 20% basic, 40% technical/implementation, 20% advanced, 20% business"""
    )

    synthetic_queries = dspy.OutputField(
        desc="""Generate exactly {num_queries} SPECIFIC, TECHNICAL search queries.

CRITICAL REQUIREMENTS:
1. Avoid generic queries like "what is X" or "why is X important"
2. Focus on IMPLEMENTATION details, tools, methods, examples
3. Include specific scenarios, use cases, or contexts
4. Reference specific tools, platforms, or technical aspects

Query distribution:
- 10% basic definitional (only 2 queries max)
- 50% TECHNICAL implementation (tools, methods, code, examples)
- 20% advanced optimization (troubleshooting, best practices)
- 20% business/comparison (pricing, services, ROI)

GOOD examples for "Content Audit":
- "How to use Google Analytics 4 to track content performance during audit"
- "Content audit metrics for e-commerce product pages"
- "Python script for automated content audit analysis"
- "Content audit checklist for 10,000+ page websites"

BAD examples (too generic):
- "What is a content audit"
- "Why content audits are important"
- "Content audit best practices"

Return ONLY valid JSON array: ["query1", "query2", ...]"""
    )


class GeminiAnalyzer:
    def __init__(self, gemini_key):
        print('🤖 Gemini setup...')
        self.client = genai.Client(api_key=gemini_key)
        self.model = GEMINI_MODEL

        os.environ['GOOGLE_API_KEY'] = gemini_key
        try:
            dspy_lm = dspy.LM(f'gemini/{GEMINI_MODEL}', api_key=gemini_key, max_tokens=2000, temperature=0.7)
            dspy.settings.configure(lm=dspy_lm)
            self.query_generator = dspy.ChainOfThought(QueryFanOutWithFacets)
            print(f'✅ DSPy configured for facets reasoning')
        except Exception as e:
            print(f'⚠️ DSPy setup failed: {e}')
            self.query_generator = None

        print(f'✅ LLM: {self.model}')
        print(f'✅ Embeddings: {GEMINI_EMBEDDING_MODEL}')

    def _generate_queries(self, entity_name, num_queries):
        """Generate queries with facets reasoning + ROBUST PARSING"""
        if not self.query_generator:
            print('⚠️ DSPy not available')
            return [], "DSPy not configured"

        current_date = datetime.datetime.now().strftime("%B %d, %Y")

        try:
            result = self.query_generator(
                entity_name=entity_name,
                current_date=current_date,
                num_queries=str(num_queries)
            )

            reasoning = result.reasoning_about_facets if hasattr(result, 'reasoning_about_facets') else "N/A"

            # AGGRESSIVE PARSING
            raw = result.synthetic_queries.strip()

            # Remove all markdown
            raw = re.sub(r'```json\s*', '', raw)
            raw = re.sub(r'```\s*', '', raw)
            raw = raw.strip()

            queries = []

            # Try 1: Direct JSON parse
            if raw.startswith('[') and raw.endswith(']'):
                try:
                    queries = json.loads(raw)
                    if isinstance(queries, list) and all(isinstance(q, str) for q in queries):
                        print(f'   ✅ Parsed via JSON')
                except:
                    pass

            # Try 2: Extract JSON array from text
            if not queries:
                match = re.search(r'\[.*?\]', raw, re.DOTALL)
                if match:
                    try:
                        queries = json.loads(match.group(0))
                        if isinstance(queries, list) and all(isinstance(q, str) for q in queries):
                            print(f'   ✅ Parsed via regex + JSON')
                    except:
                        pass

            # Try 3: Line-by-line with quote cleaning
            if not queries:
                lines = raw.split('\n')
                for line in lines:
                    line = line.strip()

                    # Skip empty, brackets, or markdown
                    if not line or line in ['[', ']', '```', '```json']:
                        continue

                    # Remove numbering: "1. " or "- " or "* "
                    line = re.sub(r'^\d+[\.\)]\s*', '', line)
                    line = re.sub(r'^[-*•]\s*', '', line)

                    # Remove trailing comma
                    line = line.rstrip(',').strip()

                    # Remove quotes if present
                    if (line.startswith('"') and line.endswith('"')) or \
                       (line.startswith("'") and line.endswith("'")):
                        line = line[1:-1]

                    if line and len(line) > 5:  # Minimum query length
                        queries.append(line)

                if queries:
                    print(f'   ✅ Parsed via line-by-line ({len(queries)} queries)')

            # Cleanup and limit
            queries = [q.strip().strip('"').strip("'") for q in queries if q.strip()]
            queries = [q for q in queries if len(q) > 5 and not q.startswith('[') and not q.startswith('{')][:num_queries]

            print(f'\n🧠 FACETS REASONING:\n{reasoning}\n')

            return queries, reasoning

        except Exception as e:
            print(f'⚠️ Query generation error: {e}')
            import traceback
            traceback.print_exc()
            return [], f"Error: {e}"

    def _extract_entity(self, title, content):
        """Extract entity using Gemini"""
        prompt = f"""Extract the main entity/topic.

Title: {title}
Content: {content[:500]}

Return JSON:
{{"entity_name": "main topic", "reasoning": "why"}}"""

        try:
            response = self.client.models.generate_content(
                model=self.model,
                contents=prompt,
                config=types.GenerateContentConfig(temperature=0.3)
            )
            result = response.text.strip().replace('```json', '').replace('```', '').strip()
            return json.loads(result)
        except:
            return {'entity_name': title, 'reasoning': 'Fallback'}

    def _embed(self, texts):
        """Embeddings with Gemini"""
        embeddings = []
        for text in texts:
            result = self.client.models.embed_content(
                model=GEMINI_EMBEDDING_MODEL,
                contents=text
            )
            embeddings.append(result.embeddings[0].values)
        return np.array(embeddings)

    def analyze(self, url, original_query=None, mode='AI Mode (complex)', threshold=0.65, use_semantic_chunking=True):
        """Full analysis with facets-based query generation + semantic chunking"""
        print(f'\n{"="*70}\n🔍 {url}\n{"="*70}\n')

        # Step 1: Extract content
        print('📄 Step 1: Extraction...')
        cd = extract_content_from_url(url)
        if not cd['success']:
            return {'success': False, 'error': cd['error'], 'url': url}
        print(f'   ✅ {cd["word_count"]} words')

        # Step 2: Extract entity
        print('\n🎯 Step 2: Entity...')
        ed = self._extract_entity(cd['title'], cd['content'])
        print(f'   ✅ {ed["entity_name"]}')

        # Step 3: Generate queries WITH FACETS REASONING
        print('\n🌐 Step 3: Query Generation (Facets Approach)...')
        min_q = MIN_QUERIES_SIMPLE if 'simple' in mode.lower() else MIN_QUERIES_COMPLEX
        queries, reasoning = self._generate_queries(ed["entity_name"], min_q)

        if not queries:
            print('   ❌ No queries generated')
            return {'success': False, 'error': 'No queries generated', 'url': url}

        print(f'   ✅ {len(queries)} queries generated')
        print('\n📋 GENERATED QUERIES:')
        for i, q in enumerate(queries, 1):
            print(f'   {i}. {q}')

        # Step 4: SEMANTIC CHUNKING (iPullRank approach)
        print('\n📦 Step 4: Chunking...')
        if use_semantic_chunking:
            try:
                chunks = semantic_chunk_text(
                    cd['content'],
                    analyzer=self,
                    min_chunk_size=3,
                    max_chunk_size=15,
                    similarity_threshold=0.75
                )
                print(f'   ✅ {len(chunks)} semantic chunks created')
            except Exception as e:
                print(f'   ⚠️ Semantic chunking failed: {e}')
                print(f'   🔄 Falling back to mechanical chunking...')
                chunks = chunk_text(cd['content'], CHUNK_SIZE, CHUNK_OVERLAP)
                print(f'   ✅ {len(chunks)} mechanical chunks')
        else:
            chunks = chunk_text(cd['content'], CHUNK_SIZE, CHUNK_OVERLAP)
            print(f'   ✅ {len(chunks)} mechanical chunks')

        # Step 5: Embeddings
        print('\n🧮 Step 5: Embeddings...')
        chunk_emb = self._embed(chunks)
        print(f'   ✅ Chunks encoded')

        # Step 6: Similarity scoring
        print('\n🎯 Step 6: Similarity scoring...')
        results = []
        covered = 0
        chunk_usage = {}  # Track which chunks are used

        for i, qt in enumerate(queries, 1):
            if not qt:
                continue

            qe = self._embed([qt])[0]
            sim = np.dot(chunk_emb, qe) / (np.linalg.norm(chunk_emb, axis=1) * np.linalg.norm(qe))
            ms = float(np.max(sim))
            bi = int(np.argmax(sim))
            cov = ms >= threshold

            if cov:
                covered += 1

            # Track chunk usage
            if bi not in chunk_usage:
                chunk_usage[bi] = 0
            chunk_usage[bi] += 1

            results.append({
                'query': qt,
                'max_similarity': round(ms, 4),
                'best_chunk_idx': bi,
                'covered': cov
            })

            print(f'   {"✅" if cov else "❌"} {i}. {qt[:40]}... {ms:.3f} [chunk #{bi}]')

        total = len(results)
        score = (covered / total * 100) if total else 0

        print(f'\n{"="*70}\n📊 {score:.2f}% ({covered}/{total})\n{"="*70}\n')

        # Chunk usage analysis
        print('📊 CHUNK USAGE ANALYSIS:')
        for chunk_idx in sorted(chunk_usage.keys()):
            usage_count = chunk_usage[chunk_idx]
            percentage = (usage_count / total * 100) if total else 0
            print(f'   Chunk #{chunk_idx}: used {usage_count}x ({percentage:.1f}%)')

        unused_chunks = set(range(len(chunks))) - set(chunk_usage.keys())
        if unused_chunks:
            print(f'\n   ⚠️ Unused chunks: {sorted(unused_chunks)}')
            print(f'   → Possible irrelevant content or missing query coverage')

        return {
            'success': True,
            'url': url,
            'timestamp': datetime.datetime.now(datetime.UTC).isoformat(),
            'entity': ed,
            'content': {
                'title': cd['title'],
                'word_count': cd['word_count'],
                'chunks_count': len(chunks),
                'chunking_method': 'semantic' if use_semantic_chunking else 'mechanical'
            },
            'query_fanout': {
                'original_query': original_query or f'What is {ed["entity_name"]}?',
                'search_mode': mode,
                'generated_count': total,
                'facets_reasoning': reasoning
            },
            'ai_visibility_score': round(score, 2),
            'covered_queries_count': covered,
            'total_queries_count': total,
            'similarity_threshold': threshold,
            'chunk_usage': chunk_usage,
            'unused_chunks': list(unused_chunks) if unused_chunks else [],
            'query_details': results,
            'models': {
                'llm': self.model,
                'embeddings': GEMINI_EMBEDDING_MODEL
            }
        }


print('✅ GeminiAnalyzer with Semantic Chunking OK')

✅ GeminiAnalyzer with Semantic Chunking OK


## 🎯 Cella 6: ESEGUI ANALISI

**Configura URL qui:**

In [50]:
URL = 'https://francescoragusa.com/content-audit/'

if GEMINI_API_KEY:
    analyzer = GeminiAnalyzer(GEMINI_API_KEY)
    results = analyzer.analyze(URL)
else:
    print('❌ Aggiungi GEMINI_API_KEY nei Secrets!')

🤖 Gemini setup...
✅ DSPy configured for facets reasoning
✅ LLM: gemini-2.0-flash-exp
✅ Embeddings: models/text-embedding-004

🔍 https://francescoragusa.com/content-audit/

📄 Step 1: Extraction...
   ✅ 3065 words

🎯 Step 2: Entity...
   ✅ Content Audit

🌐 Step 3: Query Generation (Facets Approach)...
   ✅ Parsed via JSON

🧠 FACETS REASONING:
Here are 5 key information facets for Content Audit:

*   **Definitional/Explanatory:** What is a content audit, types of content audits, key metrics involved.
*   **Practical/Implementation:** How to perform a content audit, tools used (e.g., Google Analytics, Screaming Frog, Ahrefs), step-by-step guides, templates.
*   **Comparative/Analytical:** Benefits of content audits, drawbacks, alternatives to content audits, comparing different content audit tools.
*   **Current/Temporal:** Recent trends in content auditing, updates to content audit tools as of November 2025, impact of AI on content audits.
*   **Related/Adjacent:** Content strategy, SEO a

## 📊 Cella 7: Visualizza

In [56]:
# Cella 7 - Visualizza con Tabella

try:
    if 'results' not in dir():
        print('❌ Esegui prima la Cella 6 (ESEGUI ANALISI)')
    elif not results.get('success'):
        print(f'❌ Errore analisi: {results.get("error", "Unknown")}')
    else:
        s = results['ai_visibility_score']
        c = '#4CAF50' if s >= 80 else '#FF9800' if s >= 60 else '#F44336'

        entity_name = results['entity'].get('entity_name', 'Unknown')

        # HTML Report Card
        html_content = f'''
        <div style="font-family:Arial;max-width:800px;margin:20px auto;padding:25px;border:3px solid {c};border-radius:12px">
            <h1 style="color:{c}">🔍 AI Visibility Report</h1>
            <div style="background:linear-gradient(135deg,#667eea,#764ba2);padding:40px;border-radius:10px;text-align:center;color:white;margin:20px 0">
                <div style="font-size:72px;font-weight:bold">{s}%</div>
                <p style="font-size:22px">✅ {results["covered_queries_count"]} coperte | ❌ {results["total_queries_count"]-results["covered_queries_count"]} gap</p>
                <p style="font-size:14px;opacity:0.9">Entity: {entity_name} | {results["content"]["word_count"]} words | {results["content"]["chunks_count"]} chunks</p>
            </div>
        </div>
        '''

        display(HTML(html_content))

        # TABELLA DETTAGLIATA
        print('\n' + '='*150)
        print('📊 QUERY ANALYSIS TABLE')
        print('='*150)

        # Crea DataFrame
        df = pd.DataFrame(results['query_details'])

        # Aggiungi colonna status emoji
        df['status'] = df['covered'].apply(lambda x: '✅' if x else '❌')

        # Rinomina colonne per display
        df_display = df[[
            'status',
            'query',
            'max_similarity',
            'best_chunk_idx',
            'covered'
        ]].copy()

        df_display.columns = ['Status', 'Query Sintetica', 'Similarity', 'Best Chunk', 'Covered']

        # Formatta similarity
        df_display['Similarity'] = df_display['Similarity'].apply(lambda x: f'{x:.3f}')

        # Mostra tabella completa
        with pd.option_context('display.max_rows', None,
                               'display.max_columns', None,
                               'display.width', None,
                               'display.max_colwidth', 80):
            print(df_display.to_string(index=True))

        # CHUNK USAGE ANALYSIS
        print('\n' + '='*150)
        print('📦 CHUNK USAGE ANALYSIS')
        print('='*150)

        from collections import Counter
        chunk_usage = Counter(df['best_chunk_idx'])

        print(f'\n{"Chunk":<8} {"Usage Count":<15} {"Usage %":<10} {"Top Query"}')
        print('-' * 150)

        for chunk_idx in sorted(chunk_usage.keys()):
            count = chunk_usage[chunk_idx]
            percentage = (count / len(df) * 100)

            # Get top query for this chunk
            chunk_queries = df[df['best_chunk_idx'] == chunk_idx].sort_values('max_similarity', ascending=False)
            top_query = chunk_queries.iloc[0]['query'][:80] if len(chunk_queries) > 0 else 'N/A'

            print(f'#{chunk_idx:<7} {count:<15} {percentage:>6.1f}%     {top_query}')

        # Unused chunks
        all_chunks = set(range(results['content']['chunks_count']))
        used_chunks = set(chunk_usage.keys())
        unused = all_chunks - used_chunks

        if unused:
            print(f'\n⚠️  Chunks non utilizzati: {sorted(unused)}')
            print(f'   → Possibile contenuto irrilevante o gap di coverage')

        # TOP COVERED vs TOP GAPS
        print('\n' + '='*150)
        print('✅ TOP 5 QUERY COPERTE')
        print('='*150)

        covered = df[df['covered'] == True].sort_values('max_similarity', ascending=False).head(5)

        for idx, row in covered.iterrows():
            print(f"\n{idx+1}. {row['query']}")
            print(f"   📊 Similarity: {row['max_similarity']:.3f} | Chunk: #{row['best_chunk_idx']}")

        print('\n' + '='*150)
        print('❌ TOP 5 CONTENT GAPS (Opportunità)')
        print('='*150)

        gaps = df[df['covered'] == False].sort_values('max_similarity', ascending=False).head(5)

        for idx, row in gaps.iterrows():
            print(f"\n{idx+1}. {row['query']}")
            print(f"   📊 Similarity: {row['max_similarity']:.3f} (sotto {results['similarity_threshold']})")
            print(f"   📍 Chunk match: #{row['best_chunk_idx']} (insufficiente)")

        # SUMMARY
        print('\n' + '='*150)
        print('📊 EXECUTIVE SUMMARY')
        print('='*150)
        print(f'Entity:             {entity_name}')
        print(f'URL:                {results["url"]}')
        print(f'AI Visibility:      {results["ai_visibility_score"]}%')
        print(f'Coverage:           {results["covered_queries_count"]}/{results["total_queries_count"]} queries')
        print(f'Threshold:          {results["similarity_threshold"]}')
        print(f'Content:            {results["content"]["word_count"]} words')
        print(f'Chunks:             {results["content"]["chunks_count"]} ({results["content"]["chunking_method"]})')
        print(f'Models:             {results["models"]["llm"]} + {results["models"]["embeddings"]}')
        print(f'Timestamp:          {results["timestamp"]}')

        # FACETS REASONING
        if 'facets_reasoning' in results['query_fanout']:
            print('\n' + '='*150)
            print('🧠 FACETS REASONING')
            print('='*150)
            print(results['query_fanout']['facets_reasoning'])

except NameError:
    print('❌ ERRORE: Esegui prima la Cella 6 (ESEGUI ANALISI)')
except Exception as e:
    print(f'❌ ERRORE visualizzazione: {e}')
    import traceback
    traceback.print_exc()


📊 QUERY ANALYSIS TABLE
   Status                                                                         Query Sintetica Similarity  Best Chunk  Covered
0       ✅                 Content audit checklist for identifying outdated content on a SaaS blog      0.683           0     True
1       ❌      How to use Screaming Frog for identifying duplicate content during a website audit      0.638          35    False
2       ✅                      Content audit template for e-commerce websites using Google Sheets      0.754          15     True
3       ❌  Python script for automating content audit data extraction from Google Analytics 4 API      0.576          58    False
4       ✅                             Content audit metrics for measuring the ROI of blog content      0.653           0     True
5       ✅                       Content audit for improving Core Web Vitals: a step-by-step guide      0.691           0     True
6       ❌           How to identify and fix keyword cannibalizatio